In [432]:
import requests
from bs4 import BeautifulSoup
from tabulate import tabulate
import json

In [433]:
def selectChoice():

	options_map = {
		1: ('Top movies' , 'top'),
		2: ('Most Popular Movies' , 'moviemeter'),
		3: ('Top English Movies' , 'top-english-movies'),
		4: ('Top TV Shows' , 'toptv/'),
		5: ('Most Popular TV Shows' , 'tvmeter'),
		6: ('Low Rated Movies', 'bottom'),
		7: ('Top Box Office collection', 'boxoffice')
	}

	for i,option in enumerate(options_map,1):
		print("{}) {}".format(i,options_map[option][0]))

	choice = int(input('\nChoice please..\n'))

	while(choice<1 or choice>len(options_map)):
		print('Wrong choice, enter again..')
		choice = int(input('\nChoice please..\n'))


	return options_map[choice][1]

In [133]:
def fetch_movies(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup

In [431]:
def extract_movie_data(soup, choice):

  data = json.loads(soup.find('script', {'type':"application/ld+json"}).text)

  movie_data = []
  movie_names = []
  movie_ratings = []
  movie_genre = []
  movie_ratingCount = []


  for item in data['itemListElement']:
    try:
      movie_names.append(item['item']['name'])
      movie_ratings.append(item['item']['aggregateRating']['ratingValue'])
      if (choice != 'boxoffice'):
        movie_genre.append(item['item']['genre'])
      movie_ratingCount.append(item['item']['aggregateRating']['ratingCount'])
    except:
      continue;



  index = range(1, len(movie_names)+1)

  if (choice != 'boxoffice'):
    table = zip(index, movie_names, movie_ratings, movie_genre, movie_ratingCount)
    header = ["Index", "Name", "Ratings", "Genre", "Rating Count"]
    for name, rating, genre, count in zip(movie_names, movie_ratings, movie_genre, movie_ratingCount):
      movie_data.append((name, rating, genre, count))

  else:
    table = zip(index, movie_names, movie_ratings, movie_ratingCount)
    header = ["Index", "Name", "Ratings", "Rating Count"]
    for name, rating, count in zip(movie_names, movie_ratings, movie_ratingCount):
      movie_data.append((name, rating, count))



  print(tabulate(table, headers= header, tablefmt="grid"))
  return (movie_data)

In [420]:
def print_sorted_movies(movies, min_rating):
    # Sort the movies by rating in ascending order
    movies_list = list (movies)
    sorted_movies = sorted(movies_list, key=lambda x: x[1], reverse=False)

    # Filter and print movies based on minimum rating
    for movie in sorted_movies:
      if movie[1] >= min_rating:
        print(f"Movie Name: {movie[0]}, Rating: {movie[1]}")

In [422]:
# Main program
def main():
  choice = selectChoice()
  url = "https://www.imdb.com/chart/" + choice
  headers = {'Accept': '*/*', 'Connection': 'keep-alive', 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/537.36 (KHTML,like Gecko) Chrome/70.0.3538.110 Safari/537.36', 'Accept-Language':'en-US;q=0.5,en;q=0.3', 'Cache-Control': 'max-age=0', 'Upgrade-Insecure-Requests': '1'}
  soup = fetch_movies(url, headers)
  movies = extract_movie_data(soup,choice)

  #Get user input for minimum rating
  min_rating = float(input("Enter the minimum rating (e.g., 7.5): "))
  print_sorted_movies(movies, min_rating)

In [435]:
if __name__ == '__main__':
	main()

1) Top movies
2) Most Popular Movies
3) Top English Movies
4) Top TV Shows
5) Most Popular TV Shows
6) Low Rated Movies
7) Top Box Office collection

Choice please..
1
+---------+----------------------------------------------------------------------+-----------+------------------------------+----------------+
|   Index | Name                                                                 |   Ratings | Genre                        |   Rating Count |
+=========+======================================================================+===========+==============================+================+
|       1 | The Shawshank Redemption                                             |       9.3 | Drama                        |        2938867 |
+---------+----------------------------------------------------------------------+-----------+------------------------------+----------------+
|       2 | The Godfather                                                        |       9.2 | Crime, Drama          